# S2S Skill

What was the forecast skill of S2S models? 
Did grid-cell or spatial bias correction schemes improve results?

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import paraguayfloodspy.visualize as viz # custom plotting library

Define parameters

In [ ]:
%matplotlib inline
savefigs = True

Load some plotting parameters from file

In [ ]:
%run ../config/PlotParameters.py
print(extent)
print(cmap)

In [ ]:
forecasts = ['Raw', 'XLR', 'HXLR', 'PCR', 'CCA']

In [ ]:
obs_url = 'http://iridl.ldeo.columbia.edu/home/.agmunoz/.Paraguay/.Obs/.Obsrain_90thpctl/dods'
obs_exceed = xr.open_dataarray(obs_url)
obs_exceed = obs_exceed.isel(T=(obs_exceed['T'].size - 1))
hatch0 = np.ma.masked_invalid(obs_exceed.values)
hatch = np.ma.masked_less(hatch0, 0.9)
Xh, Yh = np.meshgrid(obs_exceed.X, obs_exceed.Y)

In [ ]:
ncols = len(forecasts)
fig, axes = viz.SetupAxes(ncol = ncols, nax = 2*len(forecasts), proj = ccrs.PlateCarree(), figsize = [16, 8])
for i,m in enumerate(forecasts):
    # Forecast for December 1-7 2015
    ax = axes[0, i]
    ax.set_title(m)
    url = 'http://iridl.ldeo.columbia.edu/ds%3A/home/.agmunoz/.Paraguay/.{}/.Odds_1-7Dec2015/dods'.format(m)
    ds = xr.open_dataarray(url)
    X,Y = np.meshgrid(ds.X, ds.Y)
    sub = ds.values
    sub = np.ma.masked_invalid(sub)
    sub = np.ma.masked_less(sub, 0)
    if len(sub.shape) == 3:
        sub = sub[0,:,:]
    C1 = ax.pcolormesh(X, Y, sub, cmap="Greens", transform=ccrs.PlateCarree(), vmin=0, vmax=7)
    ax.pcolor(Xh, Yh, hatch, facecolor='none', edgecolors='k')
    # 2AFC Skill Score
    ax = axes[1, i]
    url = 'http://iridl.ldeo.columbia.edu/home/.agmunoz/.Paraguay/.{}/.2AFCProbFcsts/dods'.format(m)
    ds = xr.open_dataarray(url)
    X,Y = np.meshgrid(ds.X, ds.Y)
    sub = ds.values
    sub = np.ma.masked_invalid(sub)
    sub = np.ma.masked_greater(sub, 100)
    sub = np.ma.masked_less(sub, 0)
    if len(sub.shape) == 3:
        sub = sub[0,:,:]
    C2 = ax.pcolormesh(X, Y, sub, transform=ccrs.PlateCarree(), 
                       cmap="PuOr", vmin=0, vmax=100)
    ax.pcolor(Xh, Yh, hatch, facecolor='none', edgecolors='k')

viz.FormatAxes(axes, ticks=[np.linspace(-180, 180, 73), np.linspace(-90, 90, 37)], 
               extent = [np.min(X), np.max(X), np.min(Y), np.max(Y)])
fig.tight_layout()
fig.subplots_adjust(right=0.95)
cax1 = fig.add_axes([0.97, 0.55, 0.01, 0.3])
cax2 = fig.add_axes([0.97, 0.15, 0.01, 0.3])
# Color bar for Odds Ratio
cbar1 = fig.colorbar(C1, cax = cax1)
cbar1.update_ticks()
cbar1.set_label(r'Odds Ratio', rotation=270)
cbar1.ax.get_yaxis().labelpad = 15
# Color bar for 2AFC
cbar2 = fig.colorbar(C2, cax=cax2)
cbar2.set_label(r'2AFC Skill Score', rotation=270)
cbar2.ax.get_yaxis().labelpad = 15
if savefigs:
    fig.savefig("../figs/ForecastSkill.pdf", bbox_inches="tight")